In [1]:
%pip install -q --upgrade \
  langchain langchain-community langchain-core langchain-experimental \
  langchain-groq sqlalchemy pandas python-dotenv jinja2 \
  pylatexenc tabulate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.128 requires langchain-openai<0.3.0,>=0.2.1, but you have langchain-openai 0.3.33 which is incompatible.
crewai-tools 0.62.3 requires openai<1.100.0, but you have openai 1.107.2 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llm_connector import langchain_groq_llm_connector

llm = langchain_groq_llm_connector("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","meta-llama/llama-4-maverick-17b-128e-instruct")


Connecting to Groq LLM service…
Connected to Groq model 'meta-llama/llama-4-maverick-17b-128e-instruct'.


In [26]:
from db_loader import prepare_citibike_database

#db_path, conn, run_query = prepare_citibike_database()
db_path = "/Users/alikarami/Documents/AI Workshop/Sample Data/database.sqlite"

In [27]:
import os
from IPython.display import Markdown, HTML, display
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

In [28]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [29]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT TOP (10) [death]
FROM covidtracking 
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

In [30]:
db = SQLDatabase.from_uri(f'sqlite:///{db_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [31]:
# Sample question to test the agent with: ""Give me the total number of trips and the average duration based on each subscription type.""
QUESTION = """
"Give me the total number of trips and the average duration based on each subscription type."
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True,
)

In [32]:
agent_executor_SQL.invoke(QUESTION)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: station, status, trip, weatherThe tables in the database are station, status, trip, and weather. The question is asking about trips and subscription types, so the most relevant table is likely "trip".

Action: sql_db_schema
Action Input: trip
CREATE TABLE trip (
	id INTEGER, 
	duration INTEGER, 
	start_date TEXT, 
	start_station_name TEXT, 
	start_station_id INTEGER, 
	end_date TEXT, 
	end_station_name TEXT, 
	end_station_id INTEGER, 
	bike_id INTEGER, 
	subscription_type TEXT, 
	zip_code INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from trip table:
id	duration	start_date	start_station_name	start_station_id	end_date	end_station_name	end_station_id	bike_id	subscription_type	zip_code
4069	174	8/29/2013 9:08	2nd at South Park	64	8/29/2013 9:11	2nd at South Park	64	288	Subscriber	94114
4073	1067	8/29/2013 9:24	South Van Ness at Market	66	8/29/2013 9:42	San Francisco Caltrain 2 (330 Townsend)	69	321	Subscriber

{'input': '\n"Give me the total number of trips and the average duration based on each subscription type."\n',
 'output': "For 'Customer' subscription type, there were 103213 trips with an average duration of 3951.76 seconds. For 'Subscriber' subscription type, there were 566746 trips with an average duration of 590.05 seconds.\n\nExplanation:\nI queried the `trip` table to get the total number of trips and the average duration for each subscription type. The query grouped the results by `subscription_type` and limited the output to 30 results (though there were only 2 subscription types). The query used was:\n\n```sql\nSELECT subscription_type, COUNT(*) as total_trips, AVG(duration) as avg_duration \nFROM trip \nGROUP BY subscription_type \nLIMIT 30\n```\n\nThis query correctly returned the total number of trips and the average duration for 'Customer' and 'Subscriber' subscription types."}